In [ ]:
import sys
import replicate
import requests
import json
from dotenv import load_dotenv
import os
import base64
import IPython.display as display
from IPython.display import Image
from PIL import Image
import openai
from custom_modules import LicenseDatabase
load_dotenv()
REPLICATE_API_TOKEN = os.getenv("REPLICATE_API_TOKEN")
os.environ['REPLICATE_API_TOKEN'] = REPLICATE_API_TOKEN

In [ ]:
api_endpoint = "https://replicate.com/yorickvp/llava-13b/api?tab=python"
headers = {"Content-Type": "application/json"}
db = LicenseDatabase("../data/scraped/state_licenses_db")

In [ ]:
images_dir = "..data/images"
states_dir = "../data/images/states"
alabama_folder = "../data/images/Alabama drivers license"
image_path= "../images/dl_licenses_by_state/Alabama drivers license/original_635733316643777774-aladriverlicense.jpg"
sample_image = display.Image(filename = image_path) # only  converted so that it actually displays
sample_image

In [ ]:
def extract_text(image_path):
    output = replicate.run(
        "yorickvp/llava-13b:2facb4a474a0462c15041b78b1ad70952ea46b5ec6ad29583c0b29dbd4249591",
        input={
            "image": open(image_path, "rb"),
            "prompt": """ Use the text from the image to fill in the values for each category within the JSON Schemas.
                For each category, rate you}n.
                If a category is not present, write the value as "None"
                If a category is present but you cannot extract the value, write the value as "Unknown Value"
                
                Remember to look for non-character symbols that can indicate organ donor or veteran status.
                A circle that contains a star indicates that it is a 'Real ID'.
                Replace the "string", "boolean" and "integer" values with the extracted text.
                                
                {\
                    "name": {\
                        "first": {"type":"string"},\
                        "middle": {"type":"string"},\
                        "last": {"type":"string"}}\
                    },\
                    "address": {\
                        "street": {"type":"string"},\
                        "city": {"type":"string"},\
                        "state": {"type":"string"},\
                        "zip": {"type":"integer"}}\
                    },\
                    "real_id": {"type":"boolean"},\
                    "organ_donor": {"type":"boolean"},\
                    "veteran_status": {"type":"boolean"},\
                    "DD": {"type":"integer"}}\
                    "customer_number": {"type":"string"},\
                    "id_type": {"type":"string"},\
                    "issue_date": {"type":"string"},\
                    "expiration_date": {"type":"string"},\
                    "date_of_birth": {"type":"string"},\
                    "sex": {"type":"string"},\
                    "eyes": {"type":"string"},\
                    "class": {"type":"string"},\
                    "height": {"type":"float"},\
                    "endorsements": {"type":"string"},\
                    "restrictions": {"typ": "string"},\
                    "signature": {"type":"boolean"},\
                }\
                """
            }
        )
    extracted_text = ""
    for item in output:
        extracted_text += item
        print(item, end="")
    return extracted_text

In [ ]:
def list_image_files(directory):
    image_extensions = [".jpg", "jpeg", "png", "gif", "bmp"]
    image_files = []
    for filename in os.listdir(directory):
        if any(filename.lower().endswith(ext) for ext in image_extensions):
            image_files.append(os.path.join(directory, filename))
    return image_files

In [ ]:
def get_images_from(state_name):
    for dir_name in os.listdir(states_dir):
        dir_path = os.path.join(states_dir, dir_name)
        if dir_name.lower().startswith(state_name.lower()):
            return dir_path
    return None

In [ ]:
chosen_dir = get_images_from('Alabama')
if chosen_dir :
    image_paths = list_image_files(chosen_dir )
    print("image paths", image_paths)
else:
    print('state directory not found')

In [ ]:
image_paths = list_image_files(chosen_dir)
for path in image_paths:
    result = extract_text(path)

In [ ]:
result

In [ ]:
text_type = type(result)
text_type

In [ ]:
db = LicenseDatabase('../data/storage/license_database.db')

In [ ]:
db.insert_license_data(result)
customer_number = result.get('customer_number', {}).get('type')
record = db.retrieve_license_data(customer_number)
updated_data = result.copy()
db.update_license_data(customer_number, updated_data)
